**Задание**

    продумать написать тему курсового проекта
    подготовить данные для обучения
    продумать интенты на которые вы будете обучаться

**Для сдачи задания необходимо написать**

    на какую тему ваш бот будет общаться (на каких датасетах планируете обучать модели)
    написать на какие интенты вы будете обучать бота


# Решение

**Lord of the rings ChatBot**

`Задача чат-бота - общаться с людьми в тематике Властелина колец. Также чат-бот может сыграть в игру типо простого "Акинатора" - угадывать персонажа, которого загадал пользователь. Помимо этого бот может придумывать (генерировать) имена в стилистике Властелина колец.`

*Интенты:*
1. Болталка: чат-бот ведет диалог в стилистике Властелина колец
2. Игра "Угадай персонажа"
3. Генератор *имен*



In [ ]:
# Загружаем библиотеки
! pip install pymorphy2
! pip install annoy
! pip install stop_words

import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import string

import io
import chardet
import os
import codecs

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
import annoy
import gensim.models

from stop_words import get_stop_words
import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 8.2 MB 29.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b880baea4062a95ca4d32fabf26c90ba748724cd764b93447e622a6abef32fac
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 13.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=394598 sha256=44b7ae781ff7666627d07d6ad8f45ed642935f824f68dbedab70003d0053943b
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
Looking in indexes: https://pypi.org/simple, https

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Болталка: чат-бот ведет диалог в стилистике Властелина колец**

Для первой задачи - обучим нейросеть на тексте Властелина колец. Пользователь пишет сообщение в чат-бот, модель ищет максимально похожий текст в столбце "question" и выдает ответ с минимальной дистанцией.

In [ ]:
# Загружаем данные и сразу преобразуем их в токены
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
data = open("/content/lotr.txt").read()


# Предложения вопросы
tokens_q = tokenizer.tokenize(data)
print(len(tokens_q)), print(tokens_q[:3])

5204
['Мир изменился.', 'Я чувствую это в воде, чувствую в земле, ощущаю в воздухе.', 'Многое из того, что было - ушло.']


(None, None)

In [ ]:
# Предложения ответы: из получившегося списка вопросов
# формируем ответы - каждое последующее предложение является ответом для предыдущего

tokens_a = tokens_q[1:]
tokens_a.append(tokens_q[:1])
print(len(tokens_a)), print(tokens_a[:3])

5204
['Я чувствую это в воде, чувствую в земле, ощущаю в воздухе.', 'Многое из того, что было - ушло.', 'И не осталось тех, кто помнит об этом.']


(None, None)

In [ ]:
df = pd.DataFrame({'question': tokens_q, 'answer': tokens_a})
df.sample(2)

,question,answer
3441,"Да, хорошее местечко.",Говорят и обслуга отличная.
4480,Мне будет стыдно остаться в тылу.,Нам три дня ехать до Минас Тирита.


Функции для модели

In [ ]:
# Преобразование данных

morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

def preprocess_txt(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise]


def start_prep(data):
    result = []

    for text in tqdm(data):
      prep = preprocess_txt(text)
      result.append(prep)

    return result

In [ ]:
# Процедура поиска ближайших вопросов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for question in df.question:
        n_model = 0
        index_map[counter] = question
        que_prep = preprocess_txt(question)
        
        vector_model = np.zeros(300) #вектор твита

        for word in que_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1

        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор=
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(100) #из индексов создаем кластеры
    return model_index, index_map

In [ ]:
# Функция поиска ближайших соседей
def get_response(que, index, model, index_map, df):
    token = preprocess_txt(que)
    vector = np.zeros(300)
    norm = 0

    for word in token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_que = index.get_nns_by_vector(vector, 15, include_distances=True)
    
    result = pd.DataFrame({'question': [index_map[i]  for i in similar_que[0]], 
                           'distance' : similar_que[1], 'answer': [df[df.index == i].answer.values[0] for i in similar_que[0]]})
    

    return result

In [ ]:
tokens = start_prep(df.question)
modelFT = FastText(sentences=tokens, size=300, min_count=1, window=5, workers=8)
ft_index, index_map = get_model_index(modelFT.wv)

100%|██████████| 5204/5204 [00:06<00:00, 794.26it/s] 


Пример работы модели

In [ ]:
smp = 'Арагорн?'

print(smp)
# get_response(smp, ft_index, modelFT.wv, index_map, df)

# single answer
rand = random.randint(0,4)
resp = get_response(smp, ft_index, modelFT.wv, index_map, df)
print(resp.answer.values[rand])

Арагорн?
Держись!


**Игра "Угадай персонажа"**

Пользователь вводит описание персонажа, модель делает предсказание, к кому из персонажей это описание может относиться. Если модель не отгадала, пользователю предлагается дать подсказку - дополнительные факты о загаданном персонаже. Всего у модели 3 попытки на отгадывание.

In [ ]:
df2 = pd.read_excel('lotr_persons.xlsx')
df2.head(2)

,description,character
0,Арагорн на синдарине означает «почитаемый коро...,Арагорн
1,Странник/Скороход/Колоброд/Бродяжник/Шатун (ан...,Арагорн


In [ ]:
# Импорт дополнительных библиотек
from sklearn.model_selection import train_test_split
from string import punctuation
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [ ]:
# Преобразование данных
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
cats = set(df2.character.str.lower().to_list()) # исключаем из обучающих текстов имена самих персонажей
morpher = MorphAnalyzer()

def preprocess_text(txt):
    #print('raw: ', txt)
    txt = str(txt)
    #print('to str: ', txt)
    txt = "".join(c for c in txt if c not in exclude)
    #print('exclude: ', txt)
    txt = txt.lower()
    #print('lower: ', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split(' ') if word not in sw]
    #print('morpher: ', txt)
    txt = " ".join(c for c in txt if c not in cats)
    #print('wo cats: ', txt)
    return "".join(txt)

In [ ]:
tqdm.pandas()
df2['description'] = df2['description'].progress_apply(preprocess_text)

100%|██████████| 78/78 [00:00<00:00, 132.09it/s]


In [ ]:
df2['description'].sample(2)

53    следовать пророческий сон присниться фарамир б...
36    ранний версия сценарий экранизация питер джекс...
Name: description, dtype: object

In [ ]:
from nltk.tokenize import word_tokenize
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')


# Проведем токенизацию по 3-ем словам
upd_df2 = {'description': [],
           'character': []}

for char in df2.character.unique():
  data = df2[df2.character == char].description.to_list()
  for d in data:
    desc = tokenizer.tokenize(d)
    for el in desc:
      #print(el)
      words = el.split(' ')
      #print(words)
      n_min = 0
      n_max = len(words)
      
      while n_min != n_max:
        tok = [''.join(word) for word in words[n_min:n_min+3]]
        n_min += 1
        #print(tok)

        upd_df2['description'].append(preprocess_text(tok))
        upd_df2['character'].append(char)

In [ ]:
df2 = pd.DataFrame.from_dict(upd_df2)
df2.sample(2)

,description,character
58,инкогнито сражаться войско,Арагорн
1420,тихо мирно свой,Бильбо


In [ ]:
#Разбиваем на трейн и тест
df_train, df_test = train_test_split(df2, test_size=0.15, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((2504, 2), (442, 2))

In [ ]:
df_train['description'].sample(5)

374              элендил отправить
906      оружиеxa0— эльфийский меч
2131    являться внучка келеборный
726        участвовать битва хельм
1394        гондора пират получить
Name: description, dtype: object

In [ ]:
text_corpus_train = df_train['description'].values
text_corpus_test = df_test['description'].values

In [ ]:
from keras.preprocessing.text import Tokenizer

# Определяем токенайзер
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')

# Обучаем токенайзер на тренировочном наборе данных - > получаем готовый словарь
tokenizer.fit_on_texts(text_corpus_train)

In [ ]:
# На основе полученного словаря токены переводятся в их id и сформировываем последовательности sequences

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [ ]:
# всего слов в словаре
word_count = len(tokenizer.index_word) + 1

# находим максимальную длину будущей последовательности в тренировочном наборе
training_length = max([len(i.split()) for i in text_corpus_train])

word_count, training_length

(1564, 3)

In [ ]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

X_train[5]

array([ 457, 1102, 1456], dtype=int32)

In [ ]:
# Преобразование значений y
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['character']) 
test_enc_labels = le.transform(df_test['character'])
le.classes_

array(['Арагорн', 'Арвен', 'Бильбо', 'Боромир', 'Фродо'], dtype=object)

In [ ]:
num_classes = df_train['character'].nunique()
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

In [ ]:
y_train.shape, y_train[3]

((2504, 5), array([0., 0., 0., 1., 0.], dtype=float32))

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss', patience = 1, mode = 'min') 

# RNN - > CNN
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=num_classes, # !!! указать кол-во категорий
              trainable=True,
              mask_zero=True))

model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3, padding = 'same', activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) # !!! указать кол-во категорий

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
251/251 [==============================] - 8s 21ms/step - loss: 0.5448 - accuracy: 0.2460 - val_loss: 0.4865 - val_accuracy: 0.3100
Epoch 2/10
251/251 [==============================] - 5s 20ms/step - loss: 0.5066 - accuracy: 0.2696 - val_loss: 0.4726 - val_accuracy: 0.3100
Epoch 3/10
251/251 [==============================] - 5s 20ms/step - loss: 0.4361 - accuracy: 0.3614 - val_loss: 0.3830 - val_accuracy: 0.4570
Epoch 4/10
251/251 [==============================] - 5s 20ms/step - loss: 0.3762 - accuracy: 0.4497 - val_loss: 0.3571 - val_accuracy: 0.5136
Epoch 5/10
251/251 [==============================] - 5s 20ms/step - loss: 0.3313 - accuracy: 0.5579 - val_loss: 0.3039 - val_accuracy: 0.5995
Epoch 6/10
251/251 [==============================] - 5s 20ms/step - loss: 0.2735 - accuracy: 0.6817 - val_loss: 0.2883 - val_accuracy: 0.6765
Epoch 7/10
251/251 [==============================] - 5s 20ms/step - loss: 0.2326 - accuracy: 0.7396 - val_loss: 0.2372 - val_accuracy: 0.7466

In [ ]:
predictions = model.predict(X_test)

In [ ]:
preds = []
for pred in predictions:
    preds.append(pred.argmax())
    
len(preds), preds[:3]

(442, [0, 1, 2])

In [ ]:
dec_preds = le.inverse_transform(preds)
dec_preds[:3]

array(['Арагорн', 'Арвен', 'Бильбо'], dtype=object)

In [ ]:
cols = ['description', 'character']
vals = df_test[cols]
vals['preds'] = dec_preds
vals.sample(5)

,description,character,preds
259,выстлать плитка пол,Бильбо,Бильбо
327,ямочка подбородок ясный,Фродо,Фродо
86,пиппин смерть амон,Боромир,Боромир
204,дверь открываться холл,Бильбо,Бильбо
219,фродый дальнейший,Бильбо,Бильбо


Пример работы модели

In [ ]:
def prep_input_text(text):
  text = preprocess_text(text)
  text_list = []
  text_list.append(text)
  sequences = tokenizer.texts_to_sequences(text_list)
  pad = pad_sequences(sequences, maxlen=training_length)
  return pad
  
def make_pred(pad):
  predictions_single = model.predict(pad)
  preds = [predictions_single.argmax()]
  return le.inverse_transform(preds)

In [ ]:
inp = 'влюблен в эльфийку'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  влюблен в эльфийку
Ответ:  ['Арагорн']


In [ ]:
inp = 'носит кольцо'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  носит кольцо
Ответ:  ['Боромир']


In [ ]:
inp = 'Элезар'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  Элезар
Ответ:  ['Арагорн']


In [ ]:
inp = 'друг Сэма'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  друг Сэма
Ответ:  ['Фродо']


**Генератор имен**

Пользователю предлагается задать начальную букву имени, бот в ответ выдаст сгенерированное имя, похожее на эльфийское. Для этой задачи напишем простой алгоритм, который будет основываться на списке имен из Властелина колец и генерировать случайное.

In [ ]:
# Загружаем данные
df3 = pd.read_excel('lotr_names.xlsx')
df3.sample(3)

,name
4,Селебриан
26,Галадриэль
57,Идриль


In [ ]:
# Разобъем имена на фонемы и создадим словарь по первым буквам

fons = {'а': [], 'б': [], 'в': [], 'г': [], 'д': [], 'е': [], 'ё': [], 'ж': [], 'з': [], 'и': [], 'й': [], 'к': [], 'л': [], 'м': [], 'н': [],
        'о': [], 'п': [], 'р': [], 'с': [], 'т': [], 'у': [], 'ф': [], 'х': [], 'ц': [], 'ч': [], 'ш': [], 'щ': [], 'ы':[], 'ъ':[], 'ь': [], 'э': [], 'ю': [], 'я': [],
        '-': []}

for name in df3.name.str.lower().to_list():
  #print(name, len(name))
  n_min = 0
  n_max = len(name)
  while n_min != n_max:
    fon = name[n_min:n_min+3]
    fl = fon[:1]
    fons[fl].append(fon)
    n_min +=1


In [ ]:
def gen_name(letter):
  vars = fons[letter]
  r = random.randint(0, len(vars))
  name = vars[r]
  name_length = random.randint(1, 3)
  
  while name_length:
    
    try:
      nl = random.choice(list(fons.keys()))[0]
      var_nl = fons[nl]
      r_nl = random.randint(0, len(var_nl))
      name += var_nl[r_nl]
    except IndexError:
      pass

    name_length -= 1

  return name.title()

In [ ]:
let = 'о'
print('Ваше имя: ', gen_name(let))

Ваше имя:  Орнвин
